In [1]:
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
# from keras.utils import np_utils # type: ignore


In [2]:
filename = "blackfriday.txt"
raw_text = open(filename, encoding="UTF-8").read()
raw_text = raw_text.lower()

In [3]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [4]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  5515
Total Vocab:  66


In [5]:
seq_lenght = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_lenght, 1):
    seq_in = raw_text[i:i + seq_lenght]
    seq_out = raw_text[i + seq_lenght]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  5415


In [6]:
X = np.reshape(dataX, (n_patterns, seq_lenght, 1))
X = X / float(n_vocab)
y = to_categorical(dataY)

In [7]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}.keras"
checkpoint = ModelCheckpoint(filepath, monitor="loss", verbose=1, save_best_only=True, mode="min")
callbacks_list = [checkpoint]
model.fit(X, y, epochs=80, batch_size=64, callbacks=callbacks_list)

Epoch 1/80
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 3.4715
Epoch 1: loss improved from inf to 3.30783, saving model to weights-improvement-01-3.3078.keras
85/85 ━━━━━━━━━━━━━━━━━━━━ 96s 1s/step - loss: 3.4696
Epoch 2/80
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 3.2452
Epoch 2: loss improved from 3.30783 to 3.22018, saving model to weights-improvement-02-3.2202.keras
85/85 ━━━━━━━━━━━━━━━━━━━━ 140s 1s/step - loss: 3.2449
Epoch 3/80
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 3.2009
Epoch 3: loss improved from 3.22018 to 3.21347, saving model to weights-improvement-03-3.2135.keras
85/85 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step - loss: 3.2010
Epoch 4/80
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 3.1965
Epoch 4: loss improved from 3.21347 to 3.19144, saving model to weights-improvement-04-3.1914.keras
85/85 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step - loss: 3.1965
Epoch 5/80
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 3.1726
Epoch 5: loss improved from 3.19144 to 3.15592, saving model to w

import sys

int_to_char = dict((i, c) for i, c in enumerate(chars))

filename = "weights-improvement-49-0.3748.keras"
model.load_weights(filename)
start = np.random.randint(0, len(dataX) - 1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
for i in range(1000):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

In [9]:
import sys

int_to_char = dict((i, c) for i, c in enumerate(chars))

filename = "weights-improvement-72-0.0274.keras"
model.load_weights(filename)

start = np.random.randint(0, len(dataX) - 1)
pattern = dataX[start]

print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

# Tạo một danh sách để lưu kết quả dự đoán
predicted_text = []

for i in range(1000):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]

    # Lưu kết quả dự đoán vào danh sách
    predicted_text.append(result)

    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)

    pattern.append(index)
    pattern = pattern[1:len(pattern)]



print("\nDone.")


Seed:
"  day of the black friday sale. 100% of surplus discounted. sale over eod and normal prices resume 
b "
inal call black friday sale ends in 3 days – 15% off sitewide!🖤 -  
black friday sale i on 🔥 50% off right now 🥵 
the black friday sales is live !!
everything 20 % off, with up to 40% off with the 3plus1 coupon!
the sale ends on november 30!

link is in the comment below! 
black friday 50% off sale ends midnight sunday pst 💥💥💥  
black friday weekend sale
ends today 8th december

20% discount off all sale items. 



mhhsbd 
black friday sale is live🦋 up to 40% off worldwide !!!

 
last day of the black friday sale. 100% of surplus discounted. sale over eod and normal prices resume 
final call black friday sale ends in 3 days – 15% off sitewide!🖤 -  
black friday sale i on 🔥 50% off right now 🥵 
the black friday sales is live !!
everything 20 % off, with up to 40% off with the 3plus1 coupon!
the sale ends on november 30!

link is in the comment below! 
black friday 50% off sale 

In [ ]:
import tweepy

# Define keys
consumer_key = 'u8ocvdT7BWgfJPPAUf2hIxNsy'
consumer_secret = 'oICveYHz0h4eIimhhVjerRE2g1ZiojVb75fwCI1AU7Rqt08y2J'
# Access
access_token = '1144097180187164673-DJ6vhx9aw6ceoLGrgGLnB5K3jzz2WS'
access_secret = 'pW1rtGI4QsfdoiDCwC9Gkx8wLguZ0lHYfHYsHzIVzkpWj'

# Create client
client = tweepy.Client(
    consumer_key=consumer_key,
    consumer_secret=consumer_secret,
    access_token=access_token,
    access_token_secret=access_secret
)

# Ensure predicted_text is a string
predicted_text = ''.join(predicted_text)  # Join the list of characters into a single string

# Create tweet
response = client.create_tweet(text=predicted_text)

# Print response (optional, to see the result)
print(response)


In [1]:
import math

final_loss = 0.0274
perplexity = math.exp(final_loss)

print(f"Perplexity: {perplexity}")


Perplexity: 1.0277788320849788
